In [ ]:
# Importação de bibliotecas
import os
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
# Tratando os outliers

def tratar_outliers(df, colunas = None):
  """
  trata outliers para colunas numéricas

  parâmetros:
  df: dataframe a ser inspecionado
  colunas: lista de colunas ou None (aplica em todas as numéricas)

  """
  if colunas is None:
    colunas = df.select_dtypes(include = ['number']).columns

  for col in colunas:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    df[col] = np.where(
        (df[col] < limite_inferior) | (df[col] > limite_superior),
        np.nan,
        df[col]
    )
  return df

# Extraindo o mes do dataset

def extrai_mes(df):
    df = df.copy()
    df["mes"] = pd.to_datetime(df["DATA"]).dt.month  # Extrai mês diretamente
    return df

# Gerando as metricas mensais
def gerar_metricas_mensais(df):
    try:
        # Lista de colunas obrigatórias
        required_columns = [
          "PRECIPITAÇÃO TOTAL, HORÁRIO (MM)",                      # MM em maiúsculo
          "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)", # MB em maiúsculo
          "RADIACAO GLOBAL (KJ/M²)",                               # KJ e M² em maiúsculo
          "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
          "TEMPERATURA DO PONTO DE ORVALHO (°C)",
          "UMIDADE RELATIVA DO AR, HORARIA (%)",
          "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)",
          "VENTO, VELOCIDADE HORARIA (M/S)"                        # M/S em maiúsculo
        ]

        # Criar colunas faltantes com valores NaN
        for col in required_columns:
            if col not in df.columns:
                df[col] = np.nan
                print(f"Aviso: Coluna {col} ausente. Preenchida com NaN.")

        # Processamento normal
        df['date'] = pd.to_datetime(df['DATA'])
        df_agg = df.groupby(df['date'].dt.month).agg(
            cli_precip_dia_med=("PRECIPITAÇÃO TOTAL, HORÁRIO (MM)", lambda x: x.notna().sum()),
            cli_precip_total=("PRECIPITAÇÃO TOTAL, HORÁRIO (MM)", "sum"),
            cli_pressao_atm_med=("PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)", "mean"),
            cli_rad_global_med=("RADIACAO GLOBAL (KJ/M²)", "mean"),
            cli_temp_ar_med=("TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)", "mean"),
            cli_temp_orvalho_med=("TEMPERATURA DO PONTO DE ORVALHO (°C)", "mean"),
            cli_umid_rel_ar_med=("UMIDADE RELATIVA DO AR, HORARIA (%)", "mean"),
            cli_umid_rel_min_med=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "mean"),
            cli_umid_rel_min_max=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "max"),
            cli_umid_rel_min_min=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "min"),
            cli_vento_veloc_max=("VENTO, VELOCIDADE HORARIA (M/S)", "max"),
            cli_vento_veloc_med=("VENTO, VELOCIDADE HORARIA (M/S)", "mean")
        ).reset_index().rename(columns={'date': 'mes'})

        # Cálculo seguro da precipitação média
        df_agg["cli_precip_dia_med"] = df_agg["cli_precip_total"] / df_agg["cli_precip_dia_med"].replace(0, np.nan)

        # Renomeação final
        df_agg.rename(columns={
            "cli_precip_dia_med": "Precipitação Média (MM)",
            "cli_precip_total": "Precipitação Total (MM)",
            "cli_pressao_atm_med": "Pressão Média (MB)",
            "cli_rad_global_med": "Radiação Média (KJ/M²)",
            "cli_temp_ar_med": "Temperatura Média (°C)",
            "cli_temp_orvalho_med": "Temperatura Orvalho Média (°C)",
            "cli_umid_rel_ar_med": "Umidade Média (%)",
            "cli_umid_rel_min_med" : "Umidade Mínima Média (%)",
            "cli_umid_rel_min_max" : "Umidade Mínima Máxima (%)",
            "cli_umid_rel_min_min" : "Umidade Mínima Mínima (%)",
            "cli_vento_veloc_max" : "Velocidade Máxima do Vento (M/S)",
            "cli_vento_veloc_med" : "Velocidade Média do Vento (M/S)"
        }, inplace=True)

        return df_agg

    except Exception as e:
        print(f"Erro na agregação: {str(e)}")
        return None

def ler_arquivo(nome_arquivo):
    try:
        # Tentar ler com UTF-8
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='utf-8')
    except UnicodeDecodeError:
        # Tentar Latin-1 como fallback
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='latin-1')

    # Debug: Mostrar colunas originais
    # print(f"\nArquivo: {os.path.basename(nome_arquivo)}")
    # print("Colunas originais:", df.columns.tolist())


    # Mapeamento de variações de colunas de data
    data_column_variants = [
        "Data",
        "DATA (YYYY-MM-DD)",
        "DATA",
        "DATA_MEDICAO",
        "DIA"
    ]
    df.columns = df.columns.str.upper().str.strip()
    # Renomear para "DATA" se alguma variação existir
    for col in data_column_variants:
      if col in df.columns:
        df.rename(columns={col: "DATA"}, inplace=True)
        #print(f"Coluna de data padronizada: {col} -> DATA")
        break
    else:  # Fora do loop
      available_cols = df.columns.tolist()
      print(f"Colunas disponíveis: {available_cols}")
      raise ValueError(f"Nenhuma coluna de data reconhecida em {nome_arquivo}")

    #print("Colunas após renomeação:", df.columns.tolist())
    return df

In [ ]:
# Extraindo o dataframe

FILEPATH = "/content/INMET_N_AC_A104_RIO_BRANCO_01-01-2023_A_31-12-2023.CSV"

df = ler_arquivo(FILEPATH)
display(df.head(10))

FileNotFoundError: [Errno 2] No such file or directory: '/content/INMET_N_AC_A104_RIO_BRANCO_01-01-2023_A_31-12-2023.CSV'

In [ ]:
# Mostrando as colunas
display(df.columns)

In [ ]:
# Pegando o mes
df_mes = extrai_mes(df)
display(df_mes.tail(10))

In [ ]:
# Cria coluna de data para contagem correta de dias
df_mes['date'] = pd.to_datetime(df_mes['Data'])

# Agregação:
df_agg = df_mes.groupby("mes").agg(
    cli_precip_dia_med=("date", "nunique"),  # Dias únicos no mês, nunique é mais robusto pois conta os valores únicos associados a cada coluna, já o count assume que há 24 registros(pode não haver)
    cli_precip_total=("PRECIPITAÇÃO TOTAL, HORÁRIO (mm)", "sum"),
    cli_pressao_atm_med=("PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)", "mean"),
    cli_rad_global_med=("RADIACAO GLOBAL (Kj/m²)", "mean"),
    cli_temp_ar_med=("TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)", "mean"),
    cli_temp_orvalho_med=("TEMPERATURA DO PONTO DE ORVALHO (°C)", "mean"),
    cli_umid_rel_ar_med=("UMIDADE RELATIVA DO AR, HORARIA (%)", "mean"),
    cli_umid_rel_min_med=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "mean"),
    cli_umid_rel_min_max=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "max"),
    cli_umid_rel_min_min=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "min"),
    cli_vento_veloc_max=("VENTO, VELOCIDADE HORARIA (m/s)", "max"),
    cli_vento_veloc_med=("VENTO, VELOCIDADE HORARIA (m/s)", "mean")
)

# Cálculo correto da precipitação média diária:
df_agg["cli_precip_dia_med"] = df_agg["cli_precip_total"] / df_agg["cli_precip_dia_med"]

display(df_agg.head(15))

In [ ]:
display(df_agg.describe())
display(df_agg.columns)
print(df_agg.columns.size)

In [ ]:
dict_nomes = {
    "cli_precip_dia_med": "Precipitação Média (mm)",
    "cli_precip_total": "Precipitação Total (mm)",
    "cli_pressao_atm_med": "Pressão Média (atm)",
    "cli_rad_global_med": "Radiação Média (kJ/m²)",
    "cli_temp_ar_med": "Temperatura Média (°C)",
    "cli_temp_orvalho_med": "Temperatura Orvalho Média (°C)",
    "cli_umid_rel_ar_med": "Umidade Média (%)",
    "cli_umid_rel_min_med" : "Umidade Mínima Média (%)",
    "cli_umid_rel_min_max" : "Umidade Mínima Máxima (%)",
    "cli_umid_rel_min_min" : "Umidade Mínima Mínima (%)",
    "cli_vento_veloc_max" : "Velocidade Máxima do Vento (m/s)",
    "cli_vento_veloc_med" : "Velocidade Média do Vento (m/s)"
}

df_agg.rename(columns = dict_nomes, inplace = True)
display(df_agg.head(15))

In [ ]:
drive.mount('/content/drive')

ESTADOS = {
    'AC': 'Acre', 'AM': 'Amazonas', 'AP': 'Amapá', 'PA': 'Pará',
    'RO': 'Rondônia', 'RR': 'Roraima', 'TO': 'Tocantins'
}

def processar_arquivos_historico(base_path):
    """
    Processa arquivos CSV brutos e salva um arquivo por estado/ano
    """
    caminho_saida = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa03'
    os.makedirs(caminho_saida, exist_ok=True)

    # Dicionário para acumular dados por estado/ano
    dados_por_estado_ano = {}

    pastas_ano = sorted(
        [pasta for pasta in os.listdir(base_path)
         if os.path.isdir(os.path.join(base_path, pasta))
         and pasta.isdigit()
         and 2000 <= int(pasta) <= 2023],
        key=int
    )

    for pasta in pastas_ano:
        ano = int(pasta)
        caminho_pasta_ano = os.path.join(base_path, pasta)

        print(f"\nProcessando ano: {ano} | Caminho: {caminho_pasta_ano}")

        for arquivo in os.listdir(caminho_pasta_ano):
            if not arquivo.upper().endswith('.CSV'):
                continue

            try:
                caminho_completo = os.path.join(caminho_pasta_ano, arquivo)
                df = ler_arquivo(caminho_completo)
                df = tratar_outliers(df)
                df = extrai_mes(df)

                # Extrair metadados
                partes = arquivo.split('_')
                sigla_estado = partes[2]
                nome_estacao = ' '.join(partes[4:-2])

                # Adicionar colunas
                df['estado'] = ESTADOS.get(sigla_estado, 'Desconhecido')
                df['ano'] = ano
                df['estacao'] = nome_estacao  # Mantemos a estação como coluna

                # Gerar métricas mensais
                df_agg = gerar_metricas_mensais(df)

                # Agrupar por estado/ano (chave do dicionário)
                chave = (sigla_estado, ano)
                if chave not in dados_por_estado_ano:
                    dados_por_estado_ano[chave] = []
                dados_por_estado_ano[chave].append(df_agg)

            except Exception as e:
                print(f"Erro no arquivo {arquivo}: {str(e)}")
                continue

    # Salvar arquivos consolidados por estado/ano
    for (sigla_estado, ano), dataframes in dados_por_estado_ano.items():
        try:
            df_consolidado = pd.concat(dataframes, ignore_index=True)
            nome_estado = ESTADOS.get(sigla_estado, 'Desconhecido').replace(' ', '_')
            nome_saida = f"INMET_PROCESSADO_{sigla_estado}_{nome_estado}_{ano}.csv"
            caminho_final = os.path.join(caminho_saida, nome_saida)
            df_consolidado.to_csv(caminho_final, index=False)
            print(f"Arquivo salvo: {caminho_final}")

        except Exception as e:
            print(f"Erro ao salvar {sigla_estado}/{ano}: {str(e)}")

    print("\nProcessamento concluído!")
BASE_PATH = '/content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos'

df_final = processar_arquivos_historico(BASE_PATH)

if not df_final.empty:
  caminho_saida = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa03'
  df_final.to_csv(caminho_saida, index=False)
  print(f"Dataset consolidado salvo em: {caminho_saida}")
else:
  print("Nenhum arquivo foi processado.")

In [ ]:
# Unificação dos datasets de clima
drive.mount('/content/drive')

# Configurações iniciais
path = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa 3'
estados = {
    'AC': 'Acre',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'PA': 'Pará',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'TO': 'Tocantins'
}

# Listar arquivos CSV
arquivos = [f for f in os.listdir(path) if f.endswith('.csv')]

# Lista para armazenar DataFrames
dataframes = []

for arquivo in arquivos:
    try:
        # Extrair informações do nome do arquivo
        partes = arquivo.split('_')
        sigla_estado = partes[2]
        ano = partes[-1].split('.')[0]

        # Verificar se é um arquivo válido
        if sigla_estado not in estados or not ano.isdigit():
            continue

        # Carregar o CSV
        df = pd.read_csv(os.path.join(path, arquivo))

        # Adicionar colunas de ano e estado
        df['ano'] = int(ano)
        df['estado'] = estados[sigla_estado]

        # Lista de colunas para tratamento (todas exceto ano e estado)
        colunas_numericas = df.columns.drop(['mes', 'ano', 'estado'])

        # Substituir valores negativos por NaN
        for coluna in colunas_numericas:
            if df[coluna].dtype in [np.int64, np.float64]:
                df[coluna] = df[coluna].where(df[coluna] >= 0, np.nan)

        dataframes.append(df)

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {str(e)}")

# Combinar todos os DataFrames
df_final = pd.concat(dataframes, ignore_index=True)

# Reordenar colunas
colunas = ['estado', 'ano', 'mes'] + [c for c in df_final.columns if c not in ['estado', 'ano', 'mes']]
df_final = df_final[colunas]

# Salvar arquivo combinado
caminho_saida = os.path.join(path, 'DADOS_COMBINADOS.csv')
df_final.to_csv(caminho_saida, index=False)

print(f"Processo concluído! Arquivo salvo em: {caminho_saida}")
print(f"Total de registros: {len(df_final)}")
print(f"Colunas: {', '.join(df_final.columns)}")

In [ ]:
# Concatenação em uma só tabela

# Montar o Google Drive
drive.mount('/content/drive')

# Carregar os datasets
caminho_meteorologia = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/dados_combinados.csv'
caminho_mapbiomas = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/MapBiomas_merged/area_queimada_total_merged.csv'

df_met = pd.read_csv(caminho_meteorologia)
df_map = pd.read_csv(caminho_mapbiomas)

estados = {
    'AC': 'Acre',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'PA': 'Pará',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'TO': 'Tocantins'
}

# Renomear colunas do MapBiomas (incluindo a nova alteração)
df_map = df_map.rename(columns={
    '_ano': 'ano',
    '_mes': 'mes',
    '_estado': 'estado',
    'que_area_queimada': 'Quantidade de área queimada (HA)'
})

# Restante do código mantido igual
df_map = df_map.astype({'ano': int, 'mes': int})
df_met = df_met.astype({'ano': int, 'mes': int})

# Padronizar nomes dos estados no MapBiomas (caso use siglas)
df_map['estado'] = df_map['estado'].str.upper().map(
    lambda x: estados.get(x, x) if x in estados.keys() else x
)

# Fazer o merge com a coluna renomeada
df_final = pd.merge(
    df_met,
    df_map[['ano', 'mes', 'estado', 'Quantidade de área queimada (HA)']],  # Usar novo nome
    on=['ano', 'mes', 'estado'],
    how='inner'
)

# Salvar o resultado (caminho mantido)
caminho_final = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/inmet_preprocessed.csv'
df_final.to_csv(caminho_final, index=False)

print("Coluna renomeada com sucesso! Confira o resultado:")
display(df_final.head(3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Coluna renomeada com sucesso! Confira o resultado:


,estado,ano,mes,Precipitação Média (MM),Precipitação Total (MM),Pressão Média (MB),Radiação Média (KJ/M²),Temperatura Média (°C),Temperatura Orvalho Média (°C),Umidade Média (%),Umidade Mínima Média (%),Umidade Mínima Máxima (%),Umidade Mínima Mínima (%),Velocidade Máxima do Vento (M/S),Velocidade Média do Vento (M/S),Quantidade de área queimada (HA)
0,Amazonas,2000,5,0.0,0.0,1004.492026,NaN,26.237716,23.545690,85.922414,82.957845,98.0,58.0,3.7,0.924026,3828.0
1,Amazonas,2000,6,0.0,0.0,1004.887222,NaN,26.419861,23.185417,83.377778,80.301255,98.0,52.0,3.5,1.024339,9452.0
2,Amazonas,2000,7,0.0,0.0,1005.176043,NaN,25.800000,22.558205,82.476447,79.559892,98.0,47.0,3.7,1.107962,9316.0
